In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(42)

# -------------------------------
# 1) Generate sample data (sine wave sequences)
# -------------------------------
def generate_data(seq_length=10, num_samples=100):
    X, y = [], []
    for _ in range(num_samples):
        start = (torch.rand(1).item()) * 2 * math.pi  # float, not tensor
        # create time steps t = start + 0.1 * i for i in 0..seq_length-1
        t = torch.arange(seq_length, dtype=torch.float32) * 0.1 + start
        seq = torch.sin(t)  # length = seq_length
        X.append(seq[:-1])  # Input: first seq_length-1 values
        y.append(seq[-1])   # Target: last value
    X = torch.stack(X).unsqueeze(-1)  # (num_samples, seq_len-1, 1)
    y = torch.stack(y).unsqueeze(-1)  # (num_samples, 1)
    return X, y

X_train, y_train = generate_data()  # shapes: (100, 9, 1), (100, 1)

# -----------------------------------------
# 2) Define the RNN Model
# -----------------------------------------
class SimpleRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=16, output_size=1):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)  # out: (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # Take last time step
        return out

model = SimpleRNN()

# ---------------------------------------------
# 3) Define Loss and Optimizer
# ---------------------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ----------------------------
# 4) Train the RNN
# ----------------------------
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

# ----------------------------
# 5) Make a Prediction
# ----------------------------
model.eval()
test_seq = torch.sin(torch.arange(9, dtype=torch.float32) * 0.1).unsqueeze(0).unsqueeze(-1)  # (1,9,1)
with torch.no_grad():
    prediction = model(test_seq)
print(f"\nPredicted next value: {prediction.item():.6f}")


Epoch 0, Loss: 0.457441
Epoch 10, Loss: 0.020560
Epoch 20, Loss: 0.007903
Epoch 30, Loss: 0.006958
Epoch 40, Loss: 0.003880
Epoch 50, Loss: 0.000978
Epoch 60, Loss: 0.000468
Epoch 70, Loss: 0.000340
Epoch 80, Loss: 0.000175
Epoch 90, Loss: 0.000149

Predicted next value: 0.770469
